In [2]:
import tensorflow as tf
import tensorflow.contrib.seq2seq as seq2seq
from tensorflow.contrib.rnn import LSTMCell, LSTMStateTuple, GRUCell, MultiRNNCell
from tensorflow.contrib.rnn.python.ops.rnn_cell import _linear

from lib import data_utils, model_utils
from configs import model_config

import tensorflow as tf
import time
import math
import os
import sys

import numpy

In [3]:
#def __init__(self, config, use_lstm=True, forward_only=False,  attention=False ,sess=None):

config = model_config.Config()
use_lstm=True 
forward_only=False
attention=True

#config에서 모델의 설정값을 가져옵니다.
vocab_size = config.vocab_size
enc_hidden_size = config.enc_hidden_size
enc_num_layers = config.enc_num_layers
dec_hidden_size = config.dec_hidden_size
dec_num_layers = config.dec_num_layers
batch_size = config.batch_size
attention = attention


#학습 과정에서 가변 learning_rate 를 적용하기 위해서 tf.Variable 타입으로 선언합니다
learning_rate = tf.Variable(float(config.learning_rate), trainable=False)
#아래 op는 learning step 에서 두번이상 loss 가 증가헀을경우 learning_rate_decay_factor 를 곱해서
#기본값인 learning_rate_decay_factor = 0.99의 경우 본 op가 한번 실행될때마다 learning_rate 가 1% 감소시키는 효과를 가저옵니다
learning_rate_decay_op = learning_rate.assign(
    learning_rate * config.learning_rate_decay_factor)
#global_step 역시 학습 과정에서 계속 증가하여야 하는 값이므로 tf.Variable 형태로 선언하고 0으로 초기화합니다
global_step = tf.Variable(0, trainable=False)

max_gradient_norm = config.max_gradient_norm
buckets = config.buckets

#RNN의 내부 구조를 LSTM과 GRU중 선택으로 분기합니다.
if use_lstm:
    single_cell1 = LSTMCell(enc_hidden_size)
    single_cell2 = LSTMCell(dec_hidden_size)
else:
    single_cell1 = GRUCell(enc_hidden_size)
    single_cell2 = GRUCell(dec_hidden_size)

#multi layers 구조의 RNN 처리를 위한 부분 입니다. tensor 1.0 ~ 1.1 구간에서 해당 구현에 대한 이슈가 존재하여 본 예제에서는
#single layers의 구현을 가정하고 진행하겠습니다.
#seq2seq 모델의 경우 2 layers 부터 학습이 매우 어려워지는 경향을 보이며 4 layers 이상은 학습이 거의 되지 않는다 라는 경험적인 의견이 많습니다.
enc_cell = MultiRNNCell([single_cell1 for _ in range(enc_num_layers)])
dec_cell = MultiRNNCell([single_cell2 for _ in range(dec_num_layers)])

#인코딩 셀과 디코딩 셀을 self로 클래스 변수로 잡아 클래스내 다른 함수에서 이를 사용할것 입니다.
encoder_cell = enc_cell
decoder_cell = dec_cell

#실제 모델의 그래프를 생성합니다.
#_make_graph(forward_only)

#tensor sever를 정의하여 모델의 학습 과정에서 파라메터과 모델구조를 저장할 수 있도록 합니다.
#saver = tf.train.Saver(tf.global_variables())

In [4]:
#def _init_data(self):
# encoder_inputs의 placeholder로 들어오는 2차원 matrix 예시 입니다.
#[
# [36, 6, 36, 6, 14, 5, 13, 35, 739, 41, 24, 103, EOS_ID],
# [3, 5, 13, 956, 3, 227, EOS_ID, GO_ID, 142, 331, 4, 17, 8, 112, 6, 155, 3, EOS_ID] , ...
#]

encoder_inputs = tf.placeholder(shape=(None, None), dtype=tf.int32, name="encoder_inputs")
encoder_inputs_length = tf.placeholder(shape=(None,), dtype=tf.int32, name="encoder_inputs_length")

# decoder_inputs의 placeholder로 들어오는 2차원 matrix 예시 입니다.
#[
# [GO_ID, 5, 15, 33, 12, 2021, 3,2274,EOS_ID],
# [GO_ID, 142, 331, 4, 17, 8, 112, 6, 155, 3, EOS_ID] , ...
#]

decoder_inputs = tf.placeholder(shape=(None, None), dtype=tf.int32, name="decoder_inputs")
decoder_inputs_length = tf.placeholder(shape=(None,), dtype=tf.int32, name="decoder_inputs_length")

#np.array 타입의 슬라이싱을 이용하여 decoder_inputs의 GO 심볼이 위치하는 부분을 제거합니다
decoder_targets = decoder_inputs[1:, :]

#input 과 output 데이터를 버킷에 사이즈에 맞춰서 잘라냅니다
#기본값은 	model.config = buckets = [(8, 15)]로 이 경우에는 input의 경우 순서열의 최대 길이가 8 output의 경우 최대길이가 15로 잘리게 됩니다
temp_encoder_inputs = encoder_inputs[:buckets[-1][0], :]
encoder_inputs2 = temp_encoder_inputs
temp_decoder_inputs = decoder_inputs[:buckets[-1][1], :]
decoder_inputs2 = temp_decoder_inputs

# cross entropy 계산을 위해서 decoder_input 패딩된 데이터와 실제 데이터를 골라내기 위한 mask 입니다.
# getbatch 에서 받은 데이터를 주입할 placeholder
target_weights = tf.placeholder(shape=(None, None), dtype=tf.float32, name="target_weights")


## data_init

seq2seq 모델에서 입력값과 출력값(예측값)을 담당하는 encoder와 decoder의 데이터 파이프라인을 정의합니다.
placeholder 라는 구조는 lazy 하게 모델을 구성하고 차후에 데이터가 입력될 구조를 정의 합니다.

![](http://i.imgur.com/Cc22Moi.png)




In [5]:
with tf.variable_scope("embedding") as scope:
    #vocab_size * hidden_size 만큼의 크기를 가진 embedding_matrix를 생성 합니다.    
    enc_embedding_matrix = tf.get_variable(
        name="enc_embedding_matrix",
        shape=[vocab_size, enc_hidden_size],
        initializer=tf.contrib.layers.xavier_initializer(),
        dtype=tf.float32)

    dec_embedding_matrix = tf.get_variable(
        name="dec_embedding_matrix",
        shape=[vocab_size, dec_hidden_size],
        initializer=tf.contrib.layers.xavier_initializer(),
        dtype=tf.float32)

    # 연속된 단어의 index값으로 표현된 입력값을 각 인덱스의 one-hot으로 표현하고 이어서
    # embedding_vector화 하는 과정을 embedding_lookup을 통해서 쉽게 처리할 수 있습니다.

    encoder_inputs_embedded = tf.nn.embedding_lookup(
        enc_embedding_matrix, encoder_inputs2)

    decoder_inputs_embedded = tf.nn.embedding_lookup(
        dec_embedding_matrix, decoder_inputs2)


## init_embeddings
one-hot으로 표현된 단어 index를 hidden size * hidden_size 사이즈 만큼의 embedding vector로 생성 합니다.

![](http://i.imgur.com/ZOj61li.png)

In [6]:
with tf.variable_scope("encoder") as scope:
    # encoder_cell을 가지고 dynamic_rnn Layer를 생성하고
    # embedding_lookup 통해서 만든 embedding_matrix를 연결합니다.
    (encoder_outputs, encoder_state) = tf.nn.dynamic_rnn(cell=encoder_cell,
                                                                inputs=encoder_inputs_embedded,
                                                                sequence_length=encoder_inputs_length,
                                                                time_major=True, dtype=tf.float32)

## init_encoder

encoder_cell을 가지고 dynamic_rnn을 통해서 Encoder 구조를 정의 합니다.

![](http://i.imgur.com/AhmowtE.png)

In [7]:
#def _init_decoder(self, forward_only):
with tf.variable_scope("decoder") as scope:
    def output_fn(outputs):
        return tf.contrib.layers.linear(outputs, vocab_size, scope=scope)

    # attention_states: size [batch_size, max_time, num_units]
    attention_states = tf.transpose(encoder_outputs, [1, 0, 2])

    #encoder_outputs 을 가지고 attention network에 필요한 값을 생성합니다.
    (attention_keys, attention_values, attention_score_fn, attention_construct_fn) = (
        seq2seq.prepare_attention(
            attention_states=attention_states,
            attention_option="bahdanau",
            num_units=dec_hidden_size))

    #prediction 전용 모델인경우
    if forward_only:
        #decoder 함수로 inference를 사용하고
        #아래의 train 과정보다 더 많은 인자를 입력 받는데
        #전처리 과정에서 진행한 embedding 과정의 역순을 일부 자동으로 처리하기 위함입니다.
        decoder_fn = seq2seq.attention_decoder_fn_inference(
            output_fn=output_fn,
            encoder_state=encoder_state,
            attention_keys=attention_keys,
            attention_values=attention_values,
            attention_score_fn=attention_score_fn,
            attention_construct_fn=attention_construct_fn,
            embeddings=dec_embedding_matrix,
            start_of_sequence_id=model_config.GO_ID,
            end_of_sequence_id=model_config.EOS_ID,
            maximum_length=buckets[-1][1],
            num_decoder_symbols=vocab_size,
        )
        # rnn_decoder Layer를 생성합니다
        # encoder를 지나서 계산된 c는 decoder_fn의 인자를 통해 decoder에 연결되고
        # decoder_outputs에서 logit 형태로 예측값을 출력합니다.
        (decoder_outputs, decoder_state, decoder_context_state) = (
            seq2seq.dynamic_rnn_decoder(
                cell=decoder_cell,
                decoder_fn=decoder_fn,
                time_major=True,
                scope=scope,
            ))
    #train 모델인경우
    else:
        #decoder 함수로 train을 사용하고
        #이전 레이어의 출력값인 encoder_state와 함께 attention 구현에 필요한 값을 입력합니다.
        decoder_fn = seq2seq.attention_decoder_fn_train(
            encoder_state=encoder_state,
            attention_keys=attention_keys,
            attention_values=attention_values,
            attention_score_fn=attention_score_fn,
            attention_construct_fn=attention_construct_fn,
            name='attention_decoder'
        )
        # rnn_decoder Layer를 생성합니다
        # encoder를 지나서 계산된 encoder_state는 decoder_fn을 인자를 통해 decoder에 연결되고
        # loos값을 계산하기 위해서 decoder_outputs을 출력합니다.
        (decoder_outputs, decoder_state, decoder_context_state) = (
            seq2seq.dynamic_rnn_decoder(
                cell=decoder_cell,
                decoder_fn=decoder_fn,
                inputs=decoder_inputs_embedded,
                sequence_length=decoder_inputs_length,
                time_major=True,
                scope=scope,
            ))

    if forward_only:
        decoder_logits = decoder_outputs
    else:
        #
        decoder_logits = output_fn(decoder_outputs)

    #vocab 사이즈 만큼의 각각 단어의 확률값으로 표현된 리스트의 순서열을 얻고
    #argmax 연산을 통해서 최대값을 찾게 됩니다.
    decoder_prediction = tf.argmax(decoder_logits, axis=-1, name='decoder_prediction')

    #loss 계산을 위한 logit과 targets을 출력
    logits = tf.transpose(decoder_logits, [1, 0, 2])
    targets = tf.transpose(decoder_targets, [1, 0])

    if not forward_only:
        #train 모델인 경우 loss 값을 정의
        seq_loss = seq2seq.sequence_loss(logits=logits, targets=targets,weights=target_weights)


## init_decoder
decoder_cell을 가지고 dynamic_rnn Layer를 생성하고 encoder 에서 나온 encoder_state를 연결합니다.  
attention의 경우 encoder 단계에서의 각 단계의 state와 output에서 중간 단계의 값을 다시 decoder 구조에 연결합니다.  
![](http://i.imgur.com/QRdjS8z.png)  

In [8]:
#params 초기화
params = tf.trainable_variables()

gradient_norms = []
updates_list = []

#Optimizer를 통하여 학습을 진행
opt = tf.train.AdamOptimizer(learning_rate)
#위에서 진행한 모든 네트워크 구조를 지나온 결과값인 loss 에서 gradients 계산하고
gradients = tf.gradients(seq_loss, params)

#clipped_gradients를 구하고
clipped_gradients, norm = tf.clip_by_global_norm(gradients, max_gradient_norm)
#계산이나 결과에서 사용되지는 않지만 텐서보드등 에서 표현하기 위해서
gradient_norms.append(norm)
#opt.apply_gradients 를 통하여 실제 params의 업데이트를 하고
updates_list.append(opt.apply_gradients(zip(clipped_gradients, params), global_step=global_step))


## Attention Decoder

최종모델 그래프에서 optimizer(adam)에 모든 그래프 간선이 연결되는것을 확인 할 수 있습니다.

![](http://imgur.com/XKLBATg.png)

In [9]:
def step(in_session, in_encoder_inputs, in_encoder_inputs_length, in_decoder_inputs, in_decoder_inputs_length, in_target_weights):
    #placeholder 로 정의한 데이터 파이프 라인에
    #get_batch 등으로 모델 외부에서 주입할 데이터를 정의합니다.
    input_feed = {
        encoder_inputs: in_encoder_inputs,
        encoder_inputs_length: in_encoder_inputs_length,
        decoder_inputs: in_decoder_inputs,
        decoder_inputs_length: in_decoder_inputs_length,
        target_weights: in_target_weights
    }
    #학습 모델이 아니라면 출력에 필요한 OP 만 묶어서 계산하고
    if forward_only:
        output_feed = [decoder_logits, decoder_prediction, encoder_state, decoder_state]
        logits, prediction, encoder_embedding, decoder_embedding = in_session.run(output_feed, input_feed)
        return None, None, logits, prediction, encoder_embedding, decoder_embedding
    #학슴 모델이라면 updates 계산을 통해 학습을 진행 합니다.
    else:
        output_feed = [updates_list, gradient_norms, seq_loss, encoder_state, decoder_state]
        updates, gradient, loss, encoder_embedding, decoder_embedding = in_session.run(output_feed, input_feed)
        return gradient, loss, None, None, encoder_embedding, decoder_embedding



In [ ]:
import tensorflow as tf
import time
import math
import os
import sys

from lib import data_utils, model_utils
from configs import model_config


#새로운 tensorflow 세션을 생성합니다 이후 sess 라는 이름으로 호출합니다
with tf.Session() as sess:

    print("Created model with fresh parameters.")
    sess.run(tf.global_variables_initializer())

    #vacab파일의 경로를 정의합니다. 여러 모델의 테스트를 위해서 사전 크기를 파일명뒤에 명시적으로 표시합니다
    #예를들어 8000 사이즈의 vovab 파일은 vocab8000.in 입니다.
    vocab_path = os.path.join(config.data_dir, 'vocab%d.in' % config.vocab_size)

    #학습 데이터의 경로를 정의합니다. 여러 모델의 테스트를 위해서 사전 크기를 파일명뒤에 명시적으로 표시합니다
    #예를들어 8000 사이즈의 사전으로 전처리된 학습 데이터는 chat_ids8000.in 입니다.
    train_data_path = os.path.join(config.data_dir, 'chat_ids%d.in' % config.vocab_size)

    #사전 파일을 로드합니다
    #vovab.get(word) 형태로 해당 워드에 맞는 사전 index 값을 반환합니다
    #vocab_rev(index) 향태로 해당 index에 맞는 워드를 반환 합니다
    vocab, vocab_rev = data_utils.load_vocabulary(vocab_path)

    #학습 데이터를 로드합니다.
    #[[[]]] 학습데이터는 3차원 리스트의 형태로 이루어지며
    # [
    #   [
    #       [2,34,523,5,3,2,5,6,32],
    #       [2,1,52,1,2,2,5,6,3],
    #	],
    #	[
    #       [2,34,523,5,3,2,5,6,32],
    #       [2,1,52,1,2,2,5,6,3]
    #	]
    # [
    # 커스터마이징이 필요한 경우가 아니면 기존의 구현을 활용하는것을 추천합니다.

    train_set = data_utils.read_data_chat(train_data_path, config)
    # print(train_set[0])


    #초기 변수를 세팅합니다.
    step_time, loss_val = 0.0, 0.0
    current_step = 0
    perplexity = 10000.0
    previous_losses = []

    
    while current_step < config.max_epoch and not forward_only:
        #스텝당 계산 시간을 검출하기 위해서 현재 시간을 기록하고
        start_time = time.time()
        #다중 버킷 모델 NMT 관련 모델에 관련된 옵션으로 여기서는 0으로 처리 합니다
        bucket_id = 0

        #데이터 유틸리티 함수를 이용하여 이번 스탭에서 모델 계산에 필요한 모든 배치 데이터를 가져 옵니다.
        step_encoder_inputs, step_encoder_inputs_length, step_decoder_inputs, step_decoder_inputs_length, step_target_weights = (data_utils.get_batch(train_set[bucket_id], config))

        #seq2seq 모델에 모든 데이터와 loss 값을 기반으로 경사도를 계산하고
        #옵티마이저가 loss를 줄이는 방향으로 step마다 모델의 파라메터를 업데이트 합니다.
        _, step_loss, _, _, enc_embedding, dec_embedding = step(sess, step_encoder_inputs, step_encoder_inputs_length, step_decoder_inputs, step_decoder_inputs_length, step_target_weights)

        #모델의 학습 상황을 확인하기 위해서 스탭당 시간을 계산하고
        step_time += (time.time() - start_time) / 100
        #loss의 변화를 합산 합니다.
        loss_val += step_loss / 100
        current_step += 1

        #100 step 마다 아래의 처리를 합니다
        #if current_step % 2 == 0:
        if current_step % 2 == 0:
            # loss *= config.max_state_length 		# Temporary purpose only
            # loss 를 가지고 perplexity 값을 계산합니다.
            perplexity = math.exp(loss_val) if loss_val < 300 else float('inf')
            print("global step %d learning rate %.4f step-time %.2f perplexity %.2f loss %.2f" %
                        (global_step.eval(), learning_rate.eval(), step_time, perplexity, loss_val))
            #스탭에서 두번 이상 연속으로 loss 값이 줄어들지 못한경우
            if len(previous_losses) > 2 and loss_val > max(previous_losses[-2:]):
                #learning_rate를 감소시켜 모델의 학습을 용이하게 합니다
                sess.run(learning_rate_decay_op)
            #100 step 당 로스를 시계열로 기록해 둡니다.
            previous_losses.append(loss_val)

            #100 step 마다 모델을 저장합니다.
            #checkoint_path = os.path.join(config.model_dir, "model.ckpt")
            #model.saver.save(sess, checkoint_path, global_step=model.global_step)
            step_time, loss_val = 0.0, 0.0
            sys.stdout.flush()


Created model with fresh parameters.
[[]]
3 24 119 27 974 9 3 3 17 4016 7885 47 420 88 3853 3 953 3 61 42 10 3 4 196 4
 70 6 8 152 24 5 79 324 45 3 6 56 16 5 13 123 45 7 4

  reading data line 1000
  reading data line 2000
  reading data line 3000
  reading data line 4000
  reading data line 5000
  reading data line 6000
  reading data line 7000
  reading data line 8000
  reading data line 9000
  reading data line 10000
  reading data line 11000
  reading data line 12000
  reading data line 13000
  reading data line 14000
  reading data line 15000
  reading data line 16000
  reading data line 17000
  reading data line 18000
  reading data line 19000
  reading data line 20000
  reading data line 21000
  reading data line 22000
  reading data line 23000
  reading data line 24000
  reading data line 25000
  reading data line 26000
  reading data line 27000
  reading data line 28000
  reading data line 29000
  reading data line 30000
  reading data line 31000
  reading data line 32000
  re

global step 60 learning rate 0.0009 step-time 0.00 perplexity 1.12 loss 0.11
global step 62 learning rate 0.0009 step-time 0.00 perplexity 1.11 loss 0.10
global step 64 learning rate 0.0009 step-time 0.00 perplexity 1.12 loss 0.11
global step 66 learning rate 0.0009 step-time 0.00 perplexity 1.12 loss 0.11
global step 68 learning rate 0.0009 step-time 0.00 perplexity 1.12 loss 0.12
global step 70 learning rate 0.0009 step-time 0.00 perplexity 1.12 loss 0.11
global step 72 learning rate 0.0009 step-time 0.00 perplexity 1.13 loss 0.12
global step 74 learning rate 0.0009 step-time 0.00 perplexity 1.10 loss 0.10
global step 76 learning rate 0.0009 step-time 0.00 perplexity 1.11 loss 0.10
global step 78 learning rate 0.0009 step-time 0.00 perplexity 1.13 loss 0.12
global step 80 learning rate 0.0009 step-time 0.00 perplexity 1.11 loss 0.11
global step 82 learning rate 0.0009 step-time 0.00 perplexity 1.12 loss 0.11
global step 84 learning rate 0.0009 step-time 0.00 perplexity 1.11 loss 0.10

global step 272 learning rate 0.0007 step-time 0.00 perplexity 1.10 loss 0.10
global step 274 learning rate 0.0007 step-time 0.00 perplexity 1.10 loss 0.10
global step 276 learning rate 0.0007 step-time 0.00 perplexity 1.10 loss 0.10
global step 278 learning rate 0.0007 step-time 0.00 perplexity 1.10 loss 0.10
global step 280 learning rate 0.0007 step-time 0.00 perplexity 1.09 loss 0.09
global step 282 learning rate 0.0007 step-time 0.00 perplexity 1.11 loss 0.11
global step 284 learning rate 0.0006 step-time 0.00 perplexity 1.11 loss 0.11
global step 286 learning rate 0.0006 step-time 0.00 perplexity 1.10 loss 0.10
global step 288 learning rate 0.0006 step-time 0.00 perplexity 1.12 loss 0.11
global step 290 learning rate 0.0006 step-time 0.00 perplexity 1.12 loss 0.11
global step 292 learning rate 0.0006 step-time 0.00 perplexity 1.11 loss 0.11
global step 294 learning rate 0.0006 step-time 0.00 perplexity 1.11 loss 0.10
global step 296 learning rate 0.0006 step-time 0.00 perplexity 1

global step 484 learning rate 0.0005 step-time 0.00 perplexity 1.10 loss 0.10
global step 486 learning rate 0.0005 step-time 0.00 perplexity 1.10 loss 0.10
global step 488 learning rate 0.0005 step-time 0.00 perplexity 1.12 loss 0.11
global step 490 learning rate 0.0005 step-time 0.00 perplexity 1.10 loss 0.09
global step 492 learning rate 0.0005 step-time 0.00 perplexity 1.12 loss 0.11
global step 494 learning rate 0.0005 step-time 0.00 perplexity 1.10 loss 0.10
global step 496 learning rate 0.0005 step-time 0.00 perplexity 1.10 loss 0.09
global step 498 learning rate 0.0005 step-time 0.00 perplexity 1.11 loss 0.10
global step 500 learning rate 0.0005 step-time 0.00 perplexity 1.11 loss 0.10
global step 502 learning rate 0.0005 step-time 0.00 perplexity 1.12 loss 0.11
global step 504 learning rate 0.0005 step-time 0.00 perplexity 1.10 loss 0.10
global step 506 learning rate 0.0005 step-time 0.00 perplexity 1.10 loss 0.10
global step 508 learning rate 0.0005 step-time 0.00 perplexity 1

global step 696 learning rate 0.0003 step-time 0.00 perplexity 1.10 loss 0.10
global step 698 learning rate 0.0003 step-time 0.00 perplexity 1.11 loss 0.11
global step 700 learning rate 0.0003 step-time 0.00 perplexity 1.10 loss 0.09
global step 702 learning rate 0.0003 step-time 0.00 perplexity 1.09 loss 0.09
global step 704 learning rate 0.0003 step-time 0.00 perplexity 1.10 loss 0.09
global step 706 learning rate 0.0003 step-time 0.00 perplexity 1.11 loss 0.11
global step 708 learning rate 0.0003 step-time 0.00 perplexity 1.10 loss 0.09
global step 710 learning rate 0.0003 step-time 0.00 perplexity 1.10 loss 0.10
global step 712 learning rate 0.0003 step-time 0.00 perplexity 1.11 loss 0.10
global step 714 learning rate 0.0003 step-time 0.00 perplexity 1.12 loss 0.12
global step 716 learning rate 0.0003 step-time 0.00 perplexity 1.10 loss 0.10
global step 718 learning rate 0.0003 step-time 0.00 perplexity 1.10 loss 0.10
global step 720 learning rate 0.0003 step-time 0.00 perplexity 1

...

global step 51876 learning rate 0.0000 step-time 0.00 perplexity 1.12 loss 0.11
global step 51878 learning rate 0.0000 step-time 0.00 perplexity 1.10 loss 0.09
global step 51880 learning rate 0.0000 step-time 0.00 perplexity 1.10 loss 0.10
global step 51882 learning rate 0.0000 step-time 0.00 perplexity 1.08 loss 0.08
global step 51884 learning rate 0.0000 step-time 0.00 perplexity 1.09 loss 0.08
global step 51886 learning rate 0.0000 step-time 0.00 perplexity 1.09 loss 0.09
global step 51888 learning rate 0.0000 step-time 0.00 perplexity 1.10 loss 0.09
global step 51890 learning rate 0.0000 step-time 0.00 perplexity 1.10 loss 0.10
global step 51892 learning rate 0.0000 step-time 0.00 perplexity 1.09 loss 0.09
global step 51894 learning rate 0.0000 step-time 0.00 perplexity 1.12 loss 0.11
global step 51896 learning rate 0.0000 step-time 0.00 perplexity 1.09 loss 0.09
global step 51898 learning rate 0.0000 step-time 0.00 perplexity 1.11 loss 0.10
global step 51900 learning rate 0.0000 s

global step 52082 learning rate 0.0000 step-time 0.00 perplexity 1.09 loss 0.09
global step 52084 learning rate 0.0000 step-time 0.00 perplexity 1.10 loss 0.09
global step 52086 learning rate 0.0000 step-time 0.00 perplexity 1.10 loss 0.10
global step 52088 learning rate 0.0000 step-time 0.00 perplexity 1.10 loss 0.09
global step 52090 learning rate 0.0000 step-time 0.00 perplexity 1.12 loss 0.11
global step 52092 learning rate 0.0000 step-time 0.00 perplexity 1.11 loss 0.10
global step 52094 learning rate 0.0000 step-time 0.00 perplexity 1.10 loss 0.10
global step 52096 learning rate 0.0000 step-time 0.00 perplexity 1.11 loss 0.11
global step 52098 learning rate 0.0000 step-time 0.00 perplexity 1.09 loss 0.09
global step 52100 learning rate 0.0000 step-time 0.00 perplexity 1.10 loss 0.10
global step 52102 learning rate 0.0000 step-time 0.00 perplexity 1.08 loss 0.08
global step 52104 learning rate 0.0000 step-time 0.00 perplexity 1.12 loss 0.11
global step 52106 learning rate 0.0000 s

global step 52288 learning rate 0.0000 step-time 0.00 perplexity 1.11 loss 0.10
global step 52290 learning rate 0.0000 step-time 0.00 perplexity 1.10 loss 0.09
global step 52292 learning rate 0.0000 step-time 0.00 perplexity 1.10 loss 0.10
global step 52294 learning rate 0.0000 step-time 0.00 perplexity 1.11 loss 0.10
global step 52296 learning rate 0.0000 step-time 0.00 perplexity 1.10 loss 0.09
global step 52298 learning rate 0.0000 step-time 0.00 perplexity 1.12 loss 0.11
global step 52300 learning rate 0.0000 step-time 0.00 perplexity 1.08 loss 0.08
global step 52302 learning rate 0.0000 step-time 0.00 perplexity 1.09 loss 0.09
global step 52304 learning rate 0.0000 step-time 0.00 perplexity 1.10 loss 0.10
global step 52306 learning rate 0.0000 step-time 0.00 perplexity 1.10 loss 0.10
global step 52308 learning rate 0.0000 step-time 0.00 perplexity 1.08 loss 0.08
global step 52310 learning rate 0.0000 step-time 0.00 perplexity 1.10 loss 0.10
global step 52312 learning rate 0.0000 s

global step 52494 learning rate 0.0000 step-time 0.00 perplexity 1.11 loss 0.11
global step 52496 learning rate 0.0000 step-time 0.00 perplexity 1.12 loss 0.11
global step 52498 learning rate 0.0000 step-time 0.00 perplexity 1.10 loss 0.09
global step 52500 learning rate 0.0000 step-time 0.00 perplexity 1.11 loss 0.10
global step 52502 learning rate 0.0000 step-time 0.00 perplexity 1.13 loss 0.12
global step 52504 learning rate 0.0000 step-time 0.00 perplexity 1.11 loss 0.10
global step 52506 learning rate 0.0000 step-time 0.00 perplexity 1.10 loss 0.09
global step 52508 learning rate 0.0000 step-time 0.00 perplexity 1.09 loss 0.09
global step 52510 learning rate 0.0000 step-time 0.00 perplexity 1.10 loss 0.10
global step 52512 learning rate 0.0000 step-time 0.00 perplexity 1.10 loss 0.10
global step 52514 learning rate 0.0000 step-time 0.00 perplexity 1.10 loss 0.10
global step 52516 learning rate 0.0000 step-time 0.00 perplexity 1.09 loss 0.08
global step 52518 learning rate 0.0000 s

global step 52700 learning rate 0.0000 step-time 0.00 perplexity 1.10 loss 0.10
global step 52702 learning rate 0.0000 step-time 0.00 perplexity 1.10 loss 0.09
global step 52704 learning rate 0.0000 step-time 0.00 perplexity 1.10 loss 0.10
global step 52706 learning rate 0.0000 step-time 0.00 perplexity 1.10 loss 0.09
global step 52708 learning rate 0.0000 step-time 0.00 perplexity 1.12 loss 0.11
global step 52710 learning rate 0.0000 step-time 0.00 perplexity 1.09 loss 0.09
global step 52712 learning rate 0.0000 step-time 0.00 perplexity 1.10 loss 0.10
global step 52714 learning rate 0.0000 step-time 0.00 perplexity 1.10 loss 0.09
global step 52716 learning rate 0.0000 step-time 0.00 perplexity 1.09 loss 0.09
global step 52718 learning rate 0.0000 step-time 0.00 perplexity 1.10 loss 0.09
global step 52720 learning rate 0.0000 step-time 0.00 perplexity 1.10 loss 0.09
global step 52722 learning rate 0.0000 step-time 0.00 perplexity 1.11 loss 0.10
global step 52724 learning rate 0.0000 s